# Skip-gram using Navie-Softmax- CS224n
import the neccesarry libraries

In [45]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import nltk
import random
import numpy as np
from collections import Counter
flatten = lambda l: [item for sublist in l for item in sublist]
random.seed(1024)

Check the version of torch and nltk

In [31]:
print(torch.__version__)
print(nltk.__version__)

1.0.0
3.3


## Use suitable device

In [32]:
USE_CUDA = torch.cuda.is_available()
gpus = [0]
#torch.cuda.set_device(gpus[0])
FloatTensor = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if USE_CUDA else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if USE_CUDA else torch.ByteTensor


### Define a function "getBatch" accourding to batchsize

In [33]:
def getBatch(batch_size, train_data):
    random.shuffle(train_data)
    sindex = 0
    eindex = batch_size
    while eindex < len(train_data):
        batch = train_data[sindex: eindex]
        temp = eindex
        eindex = eindex + batch_size
        sindex = temp
        yield batch
    
    if eindex >= len(train_data):
        batch = train_data[sindex:]
        yield batch

### prepare sequences and words

In [34]:
def prepare_sequence(seq, word2index):
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index["<UNK>"], seq))
    return Variable(LongTensor(idxs))

def prepare_word(word, word2index):
    return Variable(LongTensor([word2index[word]]) if word2index.get(word) is not None else LongTensor([word2index["<UNK>"]]))

# Download gutenberg text files 

In [35]:
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to /home/bharat/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

## Check the corpus .txt files

In [38]:
nltk.corpus.gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

Download punkt package used for tokens creations

In [39]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/bharat/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Choose one .txt file and print list of corpus upto 100 for testing

In [48]:
corpus = list(nltk.corpus.gutenberg.sents('melville-moby_dick.txt'))[:100] # sampling sentences for test
corpus = [[word.lower() for word in sent] for sent in corpus]

In [49]:
print(corpus)

[['[', 'moby', 'dick', 'by', 'herman', 'melville', '1851', ']'], ['etymology', '.'], ['(', 'supplied', 'by', 'a', 'late', 'consumptive', 'usher', 'to', 'a', 'grammar', 'school', ')'], ['the', 'pale', 'usher', '--', 'threadbare', 'in', 'coat', ',', 'heart', ',', 'body', ',', 'and', 'brain', ';', 'i', 'see', 'him', 'now', '.'], ['he', 'was', 'ever', 'dusting', 'his', 'old', 'lexicons', 'and', 'grammars', ',', 'with', 'a', 'queer', 'handkerchief', ',', 'mockingly', 'embellished', 'with', 'all', 'the', 'gay', 'flags', 'of', 'all', 'the', 'known', 'nations', 'of', 'the', 'world', '.'], ['he', 'loved', 'to', 'dust', 'his', 'old', 'grammars', ';', 'it', 'somehow', 'mildly', 'reminded', 'him', 'of', 'his', 'mortality', '.'], ['"', 'while', 'you', 'take', 'in', 'hand', 'to', 'school', 'others', ',', 'and', 'to', 'teach', 'them', 'by', 'what', 'name', 'a', 'whale', '-', 'fish', 'is', 'to', 'be', 'called', 'in', 'our', 'tongue', 'leaving', 'out', ',', 'through', 'ignorance', ',', 'the', 'letter',

In [53]:
print(flatten(corpus))

['[', 'moby', 'dick', 'by', 'herman', 'melville', '1851', ']', 'etymology', '.', '(', 'supplied', 'by', 'a', 'late', 'consumptive', 'usher', 'to', 'a', 'grammar', 'school', ')', 'the', 'pale', 'usher', '--', 'threadbare', 'in', 'coat', ',', 'heart', ',', 'body', ',', 'and', 'brain', ';', 'i', 'see', 'him', 'now', '.', 'he', 'was', 'ever', 'dusting', 'his', 'old', 'lexicons', 'and', 'grammars', ',', 'with', 'a', 'queer', 'handkerchief', ',', 'mockingly', 'embellished', 'with', 'all', 'the', 'gay', 'flags', 'of', 'all', 'the', 'known', 'nations', 'of', 'the', 'world', '.', 'he', 'loved', 'to', 'dust', 'his', 'old', 'grammars', ';', 'it', 'somehow', 'mildly', 'reminded', 'him', 'of', 'his', 'mortality', '.', '"', 'while', 'you', 'take', 'in', 'hand', 'to', 'school', 'others', ',', 'and', 'to', 'teach', 'them', 'by', 'what', 'name', 'a', 'whale', '-', 'fish', 'is', 'to', 'be', 'called', 'in', 'our', 'tongue', 'leaving', 'out', ',', 'through', 'ignorance', ',', 'the', 'letter', 'h', ',', 'w

In [43]:
word_count = Counter(flatten(corpus))
border = int(len(word_count) * 0.01) 

In [51]:
print(word_count)

Counter({',': 96, '.': 66, 'the': 58, 'of': 36, 'and': 35, '--': 27, '"': 26, '."': 26, 'to': 25, '-': 24, 'a': 21, 'in': 20, "'": 20, 's': 19, 'his': 17, 'that': 17, 'whale': 13, ';': 12, 'is': 12, 'by': 10, 'he': 10, 'with': 10, 'or': 10, 'for': 10, 'it': 9, 'which': 9, 'great': 9, 'this': 8, 'as': 8, 'leviathan': 8, 'sub': 7, 'whales': 7, 'be': 6, 'sea': 6, 'him': 5, 'all': 5, 'take': 5, '...': 5, 'up': 5, 'but': 5, 'one': 5, '!': 5, 'ye': 5, 'had': 5, 'was': 4, 'what': 4, 'out': 4, 'not': 4, 'from': 4, 'nuee': 4, 'extracts': 4, 'are': 4, 'shall': 4, 'your': 4, 'there': 4, 'on': 4, 'like': 4, '(': 3, ')': 3, 'i': 3, 'ever': 3, 'old': 3, 'world': 3, 'you': 3, 'called': 3, 'maketh': 3, 'more': 3, 'will': 3, 'poor': 3, 'have': 3, 'long': 3, 'whom': 3, 'would': 3, 'much': 3, 'down': 3, 'lord': 3, 'mouth': 3, 'two': 3, 'other': 3, 'us': 3, 'into': 3, 'some': 3, 'king': 3, 'an': 3, 'supplied': 2, 'usher': 2, 'school': 2, 'pale': 2, 'now': 2, 'grammars': 2, 'nations': 2, 'them': 2, 'fish':

In [52]:
print(len(word_count))

592


Take 5 most common stopwords

In [67]:
stopwords = word_count.most_common()[:border] 

In [68]:
stopwords

[(',', 96), ('.', 66), ('the', 58), ('of', 36), ('and', 35)]

Most unique words

In [69]:
revstopwords= list(reversed(word_count.most_common()))[:border]

In [70]:
revstopwords

[('man', 1), ('artificial', 1), ('civitas', 1), ('--(', 1), ('state', 1)]

## Extract Stopwords from unigram distribution's tails || Most used and least used are stopwords

In [71]:
stopwords = word_count.most_common()[:border] + list(reversed(word_count.most_common()))[:border]

In [72]:
stopwords

[(',', 96),
 ('.', 66),
 ('the', 58),
 ('of', 36),
 ('and', 35),
 ('man', 1),
 ('artificial', 1),
 ('civitas', 1),
 ('--(', 1),
 ('state', 1)]

In [73]:
stopwords = [s[0] for s in stopwords]

In [74]:
stopwords

[',', '.', 'the', 'of', 'and', 'man', 'artificial', 'civitas', '--(', 'state']

# Build Vocab

In [75]:
vocab = list(set(flatten(corpus)) - set(stopwords))
vocab.append('<UNK>')

In [126]:
vocab[0:20]

['bruise',
 'baleine',
 'school',
 'ever',
 'through',
 'veritable',
 'mockingly',
 'hvalt',
 'brain',
 'six',
 'art',
 'erromangoan',
 'which',
 'immediately',
 'catched',
 'prepared',
 'alone',
 'ancient',
 'they',
 'ceti']

In [77]:
print(len(set(flatten(corpus))), len(vocab))

592 583


In [81]:
word2index = {'<UNK>' : 0} 

for vo in vocab:
    if word2index.get(vo) is None:
        word2index[vo] = len(word2index)

index2word = {v:k for k, v in word2index.items()} 

# Prepare train data

![title](training_data.png)

In [84]:
WINDOW_SIZE = 3
windows = flatten([list(nltk.ngrams(['<DUMMY>'] * WINDOW_SIZE + c + ['<DUMMY>'] * WINDOW_SIZE, WINDOW_SIZE * 2 + 1)) for c in corpus])

In [90]:
windows[0:4]

[('<DUMMY>', '<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by'),
 ('<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by', 'herman'),
 ('<DUMMY>', '[', 'moby', 'dick', 'by', 'herman', 'melville'),
 ('[', 'moby', 'dick', 'by', 'herman', 'melville', '1851')]

## Make a n-grams model
Training data will be :

In [91]:
train_data = []

for window in windows:
    for i in range(WINDOW_SIZE * 2 + 1):
        if i == WINDOW_SIZE or window[i] == '<DUMMY>': 
            continue
        train_data.append((window[WINDOW_SIZE], window[i]))

print(train_data[:WINDOW_SIZE * 2])

[('[', 'moby'), ('[', 'dick'), ('[', 'by'), ('moby', '['), ('moby', 'dick'), ('moby', 'by')]


In [92]:
X_p = []
y_p = []

In [93]:
train_data[0]

('[', 'moby')

In [94]:
for tr in train_data:
    X_p.append(prepare_word(tr[0], word2index).view(1, -1))
    y_p.append(prepare_word(tr[1], word2index).view(1, -1))

In [132]:
X_p[0:20]

[tensor([[413]]),
 tensor([[413]]),
 tensor([[413]]),
 tensor([[556]]),
 tensor([[556]]),
 tensor([[556]]),
 tensor([[556]]),
 tensor([[576]]),
 tensor([[576]]),
 tensor([[576]]),
 tensor([[576]]),
 tensor([[576]]),
 tensor([[290]]),
 tensor([[290]]),
 tensor([[290]]),
 tensor([[290]]),
 tensor([[290]]),
 tensor([[290]]),
 tensor([[50]]),
 tensor([[50]])]

In [133]:
y_p[0:20]

[tensor([[556]]),
 tensor([[576]]),
 tensor([[290]]),
 tensor([[413]]),
 tensor([[576]]),
 tensor([[290]]),
 tensor([[50]]),
 tensor([[413]]),
 tensor([[556]]),
 tensor([[290]]),
 tensor([[50]]),
 tensor([[501]]),
 tensor([[413]]),
 tensor([[556]]),
 tensor([[576]]),
 tensor([[50]]),
 tensor([[501]]),
 tensor([[58]]),
 tensor([[556]]),
 tensor([[576]])]

In [100]:
train_data = list(zip(X_p, y_p))

## Modelling

In [103]:
class Skipgram(nn.Module):
    
    def __init__(self, vocab_size, projection_dim):
        super(Skipgram,self).__init__()
        self.embedding_v = nn.Embedding(vocab_size, projection_dim)
        self.embedding_u = nn.Embedding(vocab_size, projection_dim)

        self.embedding_v.weight.data.uniform_(-1, 1) # init
        self.embedding_u.weight.data.uniform_(0, 0) # init
        #self.out = nn.Linear(projection_dim,vocab_size)
    def forward(self, center_words,target_words, outer_words):
        center_embeds = self.embedding_v(center_words) # B x 1 x D
        target_embeds = self.embedding_u(target_words) # B x 1 x D
        outer_embeds = self.embedding_u(outer_words) # B x V x D
        
        # batchwise matrix multiplication
        scores = target_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2) # Bx1xD * BxDx1 => Bx1
        norm_scores = outer_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2) # BxVxD * BxDx1 => BxV
        
        nll = -torch.mean(torch.log(torch.exp(scores)/torch.sum(torch.exp(norm_scores), 1).unsqueeze(1))) # log-softmax
        
        return nll # negative log likelihood
    
    def prediction(self, inputs):
        embeds = self.embedding_v(inputs)
        
        return embeds 

## Train

In [101]:
EMBEDDING_SIZE = 30
BATCH_SIZE = 256
EPOCH = 100

In [104]:
losses = []
model = Skipgram(len(word2index), EMBEDDING_SIZE)
if USE_CUDA:
    model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [107]:
for epoch in range(EPOCH):
    for i, batch in enumerate(getBatch(BATCH_SIZE, train_data)):
        
        inputs, targets = zip(*batch)
        
        inputs = torch.cat(inputs) # B x 1
        targets = torch.cat(targets) # B x 1
        vocabs = prepare_sequence(list(vocab), word2index).expand(inputs.size(0), len(vocab))  # B x V
        model.zero_grad()
        
        
        #negative log likelihood
        loss = model(inputs, targets, vocabs)
        
        loss.backward()
        optimizer.step()
   
        losses.append(loss.data.tolist())

    if epoch % 10 == 0:
        print("Epoch : %d, mean_loss : %.02f" % (epoch,np.mean(losses)))
        losses = []


Epoch : 0, mean_loss : 6.17
Epoch : 10, mean_loss : 4.37
Epoch : 20, mean_loss : 3.48
Epoch : 30, mean_loss : 3.31
Epoch : 40, mean_loss : 3.26
Epoch : 50, mean_loss : 3.24
Epoch : 60, mean_loss : 3.23
Epoch : 70, mean_loss : 3.21
Epoch : 80, mean_loss : 3.21
Epoch : 90, mean_loss : 3.20


In [108]:
vocabs

tensor([[  1,   2,   3,  ..., 581, 582,   0],
        [  1,   2,   3,  ..., 581, 582,   0],
        [  1,   2,   3,  ..., 581, 582,   0],
        ...,
        [  1,   2,   3,  ..., 581, 582,   0],
        [  1,   2,   3,  ..., 581, 582,   0],
        [  1,   2,   3,  ..., 581, 582,   0]])

## Test

In [110]:
def word_similarity(target, vocab):
    target_V = model.prediction(prepare_word(target, word2index))
    similarities = []
    for i in range(len(vocab)):
        if vocab[i] == target: continue
        vector = model.prediction(prepare_word(list(vocab)[i], word2index))
        cosine_sim = F.cosine_similarity(target_V, vector).data.tolist()[0] 
        similarities.append([vocab[i], cosine_sim])
    return sorted(similarities, key=lambda x: x[1], reverse=True)[:10] # sort by similarity

In [123]:
test = random.choice(list(vocab))
test

'any'

In [124]:
word_similarity(test, vocab)

[['book', 0.7919671535491943],
 ['find', 0.660162091255188],
 ['picking', 0.5818719863891602],
 ['whatsoever', 0.5688313841819763],
 ['anyways', 0.5544962882995605],
 ['vast', 0.5483270287513733],
 ['nothing', 0.5259244441986084],
 ['gulf', 0.5102213621139526],
 ['nescio', 0.5036137700080872],
 ['own', 0.4889150857925415]]